# Sensitive Set Transport Invariant Ranking (SenSTIR) demo

The idea of this notebook is to replicate the synthetic experiment shown in figure 1 of [Individually Fair Rankings](https://openreview.net/pdf?id=71zCSP_HuBN).

## Synthetic data generation

In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data.sampler import RandomSampler, BatchSampler
from torch.utils.data import IterableDataset

In [2]:
def generate_synthetic_LTR_data(majority_proportion = .8, num_queries = 100, num_docs_per_query = 10, seed=0):
    num_items = num_queries*num_docs_per_query
    X = np.random.uniform(0,3, size = (num_items,2)).astype(np.float32)
    relevance = X[:,0] + X[:,1]
    
    relevance = np.clip(relevance, 0.0,5.0)
    majority_status = np.random.choice([True, False], size=num_items, p=[majority_proportion, 1-majority_proportion])
    X[~majority_status, 1] = 0
    return [{"Q":X[i], "relevances":relevance[i], "majority_status":majority_status[i]} for i in range(num_items)]

In [3]:
class QueryIterableDataset(IterableDataset):
    '''
    iterable dataset that takes a set of items and indifintely samples sets of such items (queries) per iteration
    '''
    def __init__(self, items_dataset, shuffle, query_size):
        self.dataset = items_dataset
        self.query_size = query_size
        self.shuffle = shuffle

    def __iter__(self):
        while True:
            idx = self._infinite_indices()
            query = [self.dataset[i] for i in next(idx)]
            query = torch.utils.data.default_collate(query)
            yield query
    
    def _infinite_indices(self):
        worker_info = torch.utils.data.get_worker_info()
        seed = 0 if worker_info is None else worker_info.id
        g = torch.Generator()
        g.manual_seed(seed)
        while True:
            if self.shuffle:
                idx = (torch.randperm(len(self.dataset))[:self.query_size]).tolist()
                yield idx

In [4]:
num_docs_per_query = 10
num_queries = 100
dataset_train = generate_synthetic_LTR_data(num_queries = num_queries, num_docs_per_query = num_docs_per_query)
dataloader = torch.utils.data.DataLoader(QueryIterableDataset(dataset_train, True, num_docs_per_query), num_workers=2, batch_size=2)
#the data loader gets a batch of queries with relevance (batch x num_items_per_query) and features (batch x num_items_per_query x num_features)
next(iter(dataloader))

{'Q': tensor([[[1.8246, 0.0000],
          [0.6184, 2.7684],
          [0.7639, 0.0000],
          [1.3092, 0.0000],
          [2.0094, 2.2866],
          [2.5500, 2.3130],
          [1.1940, 2.8637],
          [1.1564, 0.4380],
          [1.8691, 0.0213],
          [0.3541, 2.5002]],
 
         [[1.6751, 1.9320],
          [1.3357, 1.0165],
          [1.6118, 2.5823],
          [2.6423, 1.0858],
          [1.9938, 2.6551],
          [2.8647, 2.2234],
          [1.4214, 1.5937],
          [0.8223, 1.5743],
          [2.6399, 0.7949],
          [0.4082, 2.7509]]]),
 'relevances': tensor([[3.4841, 3.3868, 1.6027, 3.2032, 4.2960, 4.8630, 4.0577, 1.5943, 1.8904,
          2.8543],
         [3.6071, 2.3522, 4.1941, 3.7282, 4.6489, 5.0000, 3.0151, 2.3966, 3.4348,
          3.1591]]),
 'majority_status': tensor([[False,  True, False, False,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])}

## fair distance learning

This is necessary to compute the wasserstein distance on the worst example generation (q' in the paper)

In [5]:
# we perform a logistic regression on the dataset to build a sensitive direction
from sklearn.linear_model import LogisticRegression

all_data = torch.utils.data.default_collate(dataset_train)
x = all_data['Q']
majority_status = all_data['majority_status']

LR = LogisticRegression(C = 100).fit(x, majority_status)

sens_directions = torch.tensor(LR.coef_,dtype=torch.float32).T
print('sensitive directions', sens_directions)


sensitive directions tensor([[2.2047e-02],
        [5.2825e+01]])


As we can see, the logistic regression finds a high sensitivity on the second dimension, the data generation process artificially produces this high correlation.

### Batched Wasserstein Distance

To audit the model we need to compute a Wasserstein distance between sets of items. This distance can be build by using a Mahalonobis distance as the pairwise cost between items in each set. The sensitive direction we just learned can be used to build this Mahalanobis distance.

In [6]:
from inFairness.distances import SensitiveSubspaceDistance, BatchedWassersteinDistance

In [7]:
distance_q = BatchedWassersteinDistance(SensitiveSubspaceDistance())
distance_q.fit(sens_directions)

## Model and Output distance

We also need a model we would like to train and to define a distance in the output space.

In [16]:
import torch.nn.functional as F
class MultilayerPerceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# model1 = MultilayerPerceptron()

In [9]:
from inFairness.distances import SquaredEuclideanDistance
distance_y = SquaredEuclideanDistance()
distance_y.fit(num_dims=num_docs_per_query)

It's worth nothing that in the output space we are measuring distances between sets of scores (each score corresponding to each document in a query). Therefore the dimensionality of the SquaredEuclideanDistance above.

## SenSTIR

High rho to enforce fairness.

In [39]:
from inFairness.fairalgo import SenSTIR
fairalgo1 = SenSTIR(
    network=MultilayerPerceptron(),
    distance_q=distance_q,
    distance_y=distance_y,
    rho=0.01,
    eps=0.025,
    auditor_nsteps=20,
    auditor_lr=0.1,
    monte_carlo_samples_ndcg=20,
)
fairalgo1.train()

SenSTIR(
  (network): MultilayerPerceptron(
    (fc1): Linear(in_features=2, out_features=10, bias=True)
    (fc2): Linear(in_features=10, out_features=10, bias=True)
    (fc3): Linear(in_features=10, out_features=1, bias=True)
  )
  (distance_q): BatchedWassersteinDistance(
    (distance): SensitiveSubspaceDistance()
  )
  (distance_y): SquaredEuclideanDistance()
)

## Baseline

Set rho to 0 to train the network normally.

In [11]:
from inFairness.fairalgo import SenSTIR
baseline = SenSTIR(
    network=MultilayerPerceptron(),
    distance_q=distance_q,
    distance_y=distance_y,
    rho=0.0,
    eps=1.0,
    auditor_nsteps=50,
    auditor_lr=0.05,
    monte_carlo_samples_ndcg=20,
)
baseline.train()

SenSTIR(
  (network): MultilayerPerceptron(
    (fc1): Linear(in_features=2, out_features=10, bias=True)
    (fc2): Linear(in_features=10, out_features=10, bias=True)
    (fc3): Linear(in_features=10, out_features=1, bias=True)
  )
  (distance_q): BatchedWassersteinDistance(
    (distance): SensitiveSubspaceDistance()
  )
  (distance_y): SquaredEuclideanDistance()
)

# Training Loop

In [12]:
%%time
from trainer import Trainer
trainer = Trainer(
    dataloader=dataloader,
    model=baseline,
    optimizer=torch.optim.Adam(baseline.parameters(),lr=0.005),
    max_iterations = 1000,
    print_loss_period= 20
)
trainer.train()

loss tensor(11.2935, grad_fn=<MeanBackward0>)
loss tensor(10.7321, grad_fn=<MeanBackward0>)
loss tensor(12.1320, grad_fn=<MeanBackward0>)
loss tensor(11.6465, grad_fn=<MeanBackward0>)
loss tensor(12.5944, grad_fn=<MeanBackward0>)
loss tensor(11.4085, grad_fn=<MeanBackward0>)
loss tensor(11.5727, grad_fn=<MeanBackward0>)
loss tensor(11.6411, grad_fn=<MeanBackward0>)
loss tensor(10.2405, grad_fn=<MeanBackward0>)
loss tensor(10.5633, grad_fn=<MeanBackward0>)
loss tensor(11.1581, grad_fn=<MeanBackward0>)
loss tensor(9.7829, grad_fn=<MeanBackward0>)
loss tensor(9.1914, grad_fn=<MeanBackward0>)
loss tensor(9.1587, grad_fn=<MeanBackward0>)
loss tensor(9.2823, grad_fn=<MeanBackward0>)
loss tensor(8.6258, grad_fn=<MeanBackward0>)
loss tensor(8.3198, grad_fn=<MeanBackward0>)
loss tensor(9.5528, grad_fn=<MeanBackward0>)
loss tensor(7.6492, grad_fn=<MeanBackward0>)
loss tensor(7.9255, grad_fn=<MeanBackward0>)
loss tensor(7.0945, grad_fn=<MeanBackward0>)
loss tensor(7.9413, grad_fn=<MeanBackward0>)

In [40]:
%%time
trainer = Trainer(
    dataloader=dataloader,
    model=fairalgo1,
    optimizer=torch.optim.Adam(fairalgo1.parameters(),lr=0.005),
    max_iterations = 625,
    print_loss_period= 20
)

trainer.train()

loss tensor(11.2257, grad_fn=<MeanBackward0>)
loss tensor(12.2347, grad_fn=<MeanBackward0>)
loss tensor(11.9171, grad_fn=<MeanBackward0>)
loss tensor(12.3560, grad_fn=<MeanBackward0>)
loss tensor(12.3011, grad_fn=<MeanBackward0>)
loss tensor(11.5033, grad_fn=<MeanBackward0>)
loss tensor(11.2771, grad_fn=<MeanBackward0>)
loss tensor(12.1039, grad_fn=<MeanBackward0>)
loss tensor(11.5014, grad_fn=<MeanBackward0>)
loss tensor(12.2602, grad_fn=<MeanBackward0>)
loss tensor(11.7446, grad_fn=<MeanBackward0>)
loss tensor(11.2764, grad_fn=<MeanBackward0>)
loss tensor(11.9515, grad_fn=<MeanBackward0>)
loss tensor(12.0390, grad_fn=<MeanBackward0>)
loss tensor(11.8717, grad_fn=<MeanBackward0>)
loss tensor(11.8761, grad_fn=<MeanBackward0>)
loss tensor(11.8281, grad_fn=<MeanBackward0>)
loss tensor(12.4061, grad_fn=<MeanBackward0>)
loss tensor(11.6437, grad_fn=<MeanBackward0>)
loss tensor(11.3209, grad_fn=<MeanBackward0>)
loss tensor(11.5436, grad_fn=<MeanBackward0>)
loss tensor(12.7618, grad_fn=<Mean

# Visualize Results

<Figure size 432x720 with 0 Axes>

In [41]:
from matplotlib import pyplot as plt
from matplotlib.cm import ScalarMappable

import seaborn as sns
import numpy as np

fairmodel = fairalgo1.network.eval()
baselinemodel = baseline.network.eval()

range_x = np.linspace(0,3,num=100).astype(np.float32)
range_y = np.linspace(0,3, num= 100).astype(np.float32)

def get_predicted_scores(model):
    xx, yy = np.meshgrid(range_x, range_y, sparse=False)
    y_pred = []

    for i in range(100):
        x = xx[i].reshape(-1, 1)
        y = yy[i].reshape(-1, 1)
        x_test = np.hstack([x, y])
        y_pred.append(model(torch.Tensor(x_test)).detach().numpy()[:, 0])
    
    return y_pred
        
def plot_contours_and_relevances(scores_pred, ax):
    scores_pred = np.array(scores_pred)
    ax.contourf(range_x, range_y, scores_pred)
    data_x = np.array(all_data['Q'])
    points = sns.scatterplot(x=data_x[:,0], y=data_x[:,1], c=all_data['relevances'], 
                             cmap='magma', ax=ax, alpha=0.5)

fig = plt.figure(figsize = (6,10))
fig, (ax1,ax2) = plt.subplots(1,2)
scores = get_predicted_scores(baselinemodel)
plot_contours_and_relevances(scores,ax1)
scores = get_predicted_scores(fairmodel)
plot_contours_and_relevances(scores,ax2)


# color bar denoting relevance
cmap = plt.get_cmap("magma")
norm = plt.Normalize(all_data['relevances'].min(),all_data['relevances'].max())
sm =  ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cbar = fig.colorbar(sm)

The fair model is plotted on the right. As we can see the contours of the fair model are not sensitive to the vertical dimmension while the baseline is.